In [1]:
!pip install findspark

In [2]:
import os
import sys
os.environ['SPARK_HOME']=r'/home/ritika/Downloads/spark-3.0.0-bin-hadoop2.7'

In [3]:
from __future__ import print_function
import findspark
findspark.init()
findspark.find()
import pyspark
findspark.find()
from pyspark.sql import SparkSession
from pyspark.ml.feature import HashingTF
from pyspark.ml.feature import Tokenizer
from pyspark.ml.feature import IDF

In [4]:
if __name__ == "__main__":
    spark = SparkSession\
        .builder\
        .appName("Tf-Idf")\
        .getOrCreate()

In [5]:
documentDF = spark.createDataFrame([
        (0,"Let's see an example of countVectorizer"),
        (1,"We will use pyspark library"),
        (2,"countVectorizer is important for NLP")
    ], ["index","sentence"])

In [6]:
documentDF.show(truncate=False)

+-----+---------------------------------------+
|index|sentence                               |
+-----+---------------------------------------+
|0    |Let's see an example of countVectorizer|
|1    |We will use pyspark library            |
|2    |countVectorizer is important for NLP   |
+-----+---------------------------------------+



In [7]:
tokenizer = Tokenizer(inputCol="sentence", outputCol="words")


In [8]:
wordsDf = tokenizer.transform(documentDF)

In [9]:
wordsDf.show(truncate=False)

+-----+---------------------------------------+----------------------------------------------+
|index|sentence                               |words                                         |
+-----+---------------------------------------+----------------------------------------------+
|0    |Let's see an example of countVectorizer|[let's, see, an, example, of, countvectorizer]|
|1    |We will use pyspark library            |[we, will, use, pyspark, library]             |
|2    |countVectorizer is important for NLP   |[countvectorizer, is, important, for, nlp]    |
+-----+---------------------------------------+----------------------------------------------+



In [10]:
TF = HashingTF(inputCol="words", outputCol="term_frequency")

In [11]:

tf_data = TF.transform(wordsDf)

In [12]:
tf_data.show(truncate=False)

+-----+---------------------------------------+----------------------------------------------+---------------------------------------------------------------------------+
|index|sentence                               |words                                         |term_frequency                                                             |
+-----+---------------------------------------+----------------------------------------------+---------------------------------------------------------------------------+
|0    |Let's see an example of countVectorizer|[let's, see, an, example, of, countvectorizer]|(262144,[8538,75750,143202,152540,165615,219087],[1.0,1.0,1.0,1.0,1.0,1.0])|
|1    |We will use pyspark library            |[we, will, use, pyspark, library]             |(262144,[89356,98717,133073,156084,248899],[1.0,1.0,1.0,1.0,1.0])          |
|2    |countVectorizer is important for NLP   |[countvectorizer, is, important, for, nlp]    |(262144,[15391,76307,106776,106841,152540],[1.0,1.0

In [13]:
idf = IDF(inputCol="term_frequency", outputCol="idf")

In [14]:
idf_Model = idf.fit(tf_data)

In [15]:
new_Data = idf_Model.transform(tf_data)

In [16]:
new_Data.select("index", "idf").show(truncate=False)

+-----+----------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|index|idf                                                                                                                                                                   |
+-----+----------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|0    |(262144,[8538,75750,143202,152540,165615,219087],[0.6931471805599453,0.6931471805599453,0.6931471805599453,0.28768207245178085,0.6931471805599453,0.6931471805599453])|
|1    |(262144,[89356,98717,133073,156084,248899],[0.6931471805599453,0.6931471805599453,0.6931471805599453,0.6931471805599453,0.6931471805599453])                          |
|2    |(262144,[15391,76307,106776,106841,152540],[0.6931471805599453,0.6931471805599453,0.6931471805599453,0.693147180559945

In [17]:
spark.stop()